In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the path to the folder containing ClusterAndPredict
module_path = os.path.abspath(os.path.join('..', '/workspaces/FactChecker-V3'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator
from Clustering.Helpers.Visualizer import Visualizer

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x75be052e0fd0>>
Traceback (most recent call last):
  File "/home/codespace/.local/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 
2024-10-02 14:35:57.505283: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-02 14:35:57.509893: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-02 14:35:57.522502: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 14:35:57.542482: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attemptin

In [3]:
# DEPRECATED
# chroma_client = chromadb.PersistentClient(path="./../../Clustering/Clustering/Chroma")
# # Count number of collections
# print(chroma_client.count_collections())
# 
# # Get all collection names
# collection_names = chroma_client.list_collections()
# 
# # Loop through each collection and drop it
# for collection_name in collection_names:
#     if collection_name.name != 'climate_claims_embeddings_unchanged':
#         chroma_client.delete_collection(collection_name.name)

In [4]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
print("Number of experiments to run: ", len(params))
for param in params:
    percentage = 0.75
    data_loader = DataLoader(percentage, True, param['random_seed'])
    use_only_card = param['use_only_CARD']
    size_of_dataset = param['size_of_dataset']
    del param['size_of_dataset']
    del param['use_only_CARD']

    train_df, test_df = data_loader.create_train_test_df(False, False, False, True, size_of_dataset)
    if use_only_card:
        print("using card")
        train_df, test_df = data_loader.create_train_test_df(True, True, True, True, size_of_dataset)

    clf = ClusterAndPredict(**param, train_df=train_df)
    clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
    # Print best parameters
    best_estimator = clf
    score = best_estimator.score([], [])
    print(best_estimator.get_all_performance_metrics())
    object_output = best_estimator.get_all_performance_metrics()
    cluster_df = object_output['cluster_df']
    cluster_dfs.append(cluster_df)
    output = {
        'percentage': percentage,
        'score': score,
        'accuracy': best_estimator.get_accuracy(),
        'was_supervised_umap_used': best_estimator.get_was_supervised(),
        'metrics': best_estimator.get_all_performance_metrics(),
        'size_of_dataset': size_of_dataset,
        'use_only_CARD': use_only_card
    }
    # Prepend the value 'param' to the keys in params
    for key, value in param.items():
        local_key = 'params.' + key
        output[local_key] = value

    results.append(output)

Numerical Rating
3    5189
1     253
Name: count, dtype: int64
Number of experiments to run:  1
Numerical Rating
3    5189
1     253
Name: count, dtype: int64
Numerical Rating
3    3132
Name: count, dtype: int64
Numerical Rating
3    953
1     51
Name: count, dtype: int64
using card
Numerical Rating
3    4467
1    2139
Name: count, dtype: int64
Numerical Rating
3    1044
1     829
Name: count, dtype: int64
Fitting
Getting embeddings...
Uploading claim to pinecone:  It_is_a_fact_that_the_pH_value_of_the_oceans_has_dropped_from_8._to_8..


KeyboardInterrupt: 

In [10]:
results_df = pd.json_normalize(results)
# Get current date and time
from datetime import datetime
now = datetime.now()
results_df
results_df.to_csv(f'results_${str(now)}.csv', index=False)

In [7]:
best_estimator.clusters_df
# Filter clusters_df to where cluster is not -1 and predicted_cluster is 4
best_estimator.clusters_df[(best_estimator.clusters_df['cluster'] != -1) & (best_estimator.clusters_df['predicted_veracity'] == 4)]
# Filter to cluster 144
best_estimator.clusters_df[best_estimator.clusters_df['cluster'] == 144]

,text,veracity,predict,predicted_veracity,embeddings,cluster,num_correct_in_cluster,total_in_cluster,cluster_accuracy
74,"Compared to 2022, the 2023 data show a 25% dec...",1,True,4,"[2.463346242904663, 2.7957773208618164, 9.9586...",144,0,5,0.0
378,Improvements in existing plants reduced steel-...,3,True,4,"[2.42880916595459, 2.778208017349243, 9.946139...",144,0,5,0.0
438,Reported power plant emissions increased drama...,1,True,4,"[2.5143368244171143, 2.7544777393341064, 9.951...",144,0,5,0.0
649,"In 2022, reported emissions from large industr...",1,True,4,"[2.4696571826934814, 2.7592506408691406, 9.971...",144,0,5,0.0
862,"Between 1990 and 2023, annual emissions of SO2...",1,True,4,"[2.4708685874938965, 2.7929885387420654, 9.978...",144,0,5,0.0


In [ ]:
viz = Visualizer()
df_with_two_dimens = viz.fit_transform(cluster_dfs[0], 'embeddings')
df_with_two_dimens